In [3]:
from __future__ import print_function
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Sequential, regularizers
from tensorflow.keras.layers import Layer
from tensorflow.keras import activations
from tensorflow.keras import utils
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

from tensorflow.keras.callbacks import LearningRateScheduler

import keras
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import math
from itertools import cycle
import numpy as np
import pandas as pd
import random
#import seaborn as sn
from matplotlib import pyplot as plt

from evaluation import compute_performance_measures

K.set_image_data_format('channels_last')

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  #按需分配显存
keras.backend.tensorflow_backend.set_session(tf.compat.v1.Session(config=config))

tf.random.set_seed(111111)
np.random.seed(111111)
random.seed(111111)

tf.__version__

'2.3.0'

In [4]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (1 + s_squared_norm)
    return scale * x


def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)


def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return K.sum((y_true * K.square(K.relu(1 - margin - y_pred)) + lamb * (
        1 - y_true) * K.square(K.relu(y_pred - margin))), axis=-1)


class Capsule(Layer):
    def __init__(self,
                 num_capsule,
                 dim_capsule,
                 routings=3,
                 share_weights=True,
                 activation='squash',
                 **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = activations.get(activation)
            
    def get_config(self):
        config = super().get_config().copy()
        config.update({
        'num_capsule':  self.num_capsule,
        'dim_capsule' : self.dim_capsule,
        'routings':  self.routings,
        'share_weight':self.share_weights,
           
        })
        return config

    def build(self, input_shape):
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(1, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(input_num_capsule, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)

    def call(self, inputs):
        if self.share_weights:
            hat_inputs = K.conv1d(inputs, self.kernel)
        else:
            hat_inputs = K.local_conv1d(inputs, self.kernel, [1], [1])

        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        hat_inputs = K.reshape(hat_inputs,
                               (batch_size, input_num_capsule,
                                self.num_capsule, self.dim_capsule))
        hat_inputs = K.permute_dimensions(hat_inputs, (0, 2, 1, 3))

        b = K.zeros_like(hat_inputs[:, :, :, 0])
        for i in range(self.routings):
            c = softmax(b, 1)
            o = self.activation(keras.backend.batch_dot(c, hat_inputs, [2, 2]))
            if i < self.routings - 1:
                b = keras.backend.batch_dot(o, hat_inputs, [2, 3])
                if K.backend() == 'theano':
                    o = K.sum(o, axis=1)

        return o

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [5]:
def create_model(inputs):
    x = Conv2D(64, (3, 3), activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = AveragePooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = Reshape((-1, 128))(x)
    x = Capsule(32, 8, 3, True)(x)  
    x = Capsule(32, 8, 3, True)(x)   
    capsule = Capsule(3, 16, 3, True)(x)
    output = Lambda(lambda z: K.sqrt(K.sum(K.square(z), 2)))(capsule)
    model = Model(inputs=[inputs], outputs=[output], name='COVID-CAPS')

    return model

In [6]:
inputs = Input(shape=(128, 128, 3))
model = create_model(inputs)
adam = optimizers.Adam(lr=0.0001)     
model.compile(loss=margin_loss, optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "COVID-CAPS"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 126, 126, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 64)      256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
average_pooling2d (AveragePo (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 58, 58, 128)       1

In [ ]:
#learning decay rate schedule
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5 
    epochs_drop = 20  
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

In [ ]:
batch_size = 16  
num_classes = 3
epochs = 100

images= np.load("data/image.npy")
labels= np.load("data/label.npy")

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)
cvscores = []
cvpre = []
cvrecall  =[]
cvf1 = []
cvauc = []

for k, (train, test) in enumerate(kfold.split(images, labels)):
    
    path = "model/"
    
    x_train = images[train]
    x_test = images[test]
    y_train = labels[train]
    y_test = labels[test]
    
#     np.save(path + 'data/x_train.npy', x_train)
#     np.save(path + 'data/y_train.npy', y_train)
#     np.save(path + 'data/x_test.npy', x_test)
#     np.save(path + 'data/y_test.npy', y_test)
    
    #class weights to handle class imbalance
    class_weights = {0: 1-np.count_nonzero(y_train==0)/len(y_train), 
                     1: 1-np.count_nonzero(y_train==1)/len(y_train), 
                     2: 1-np.count_nonzero(y_train==2)/len(y_train)}

    # 将整型标签转为onehot
    y_train = utils.to_categorical(y_train, num_classes)
    y_test = utils.to_categorical(y_test, num_classes)
    
    
    # The best model is selected based on the loss value on the validation set
    filepath = path+"caps-weights-best.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    
    # learning schedule callback
    lrate = LearningRateScheduler(step_decay)

    callbacks_list = [checkpoint, lrate]
    

    inputs = Input(shape=(128, 128, 3))
    #===================================Model======================================
    model = create_model(inputs)
    adam = optimizers.Adam(lr=0.0001)     
    model.compile(loss=margin_loss, optimizer=adam, metrics=['accuracy'])
    
    print("========================= 第"+ str(k+1) +"折开始 ============================")
    history = model.fit(x_train, y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=2,
                    #validation_split=0.1,
                    validation_data=(x_test, y_test),
                    class_weight=class_weights, 
                    shuffle=True, 
                    callbacks=callbacks_list)
    
#     np.save(path+'acc.npy', history.history['accuracy'])
#     np.save(path+'val_acc.npy', history.history['val_accuracy'])
#     np.save(path+'loss.npy', history.history['loss'])
#     np.save(path+'val_loss.npy', history.history['val_loss'])
    
    model.load_weights(filepath)
    # evaluate the model
    scores = model.evaluate(x_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
    predict = model.predict([x_test])
    
    # ===================auc=========================
    n_classes = y_test.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], predict[:, i], )
        roc_auc[i] = auc(fpr[i], tpr[i])
    roc_auc["macro"] = roc_auc_score(y_test, predict, multi_class="ovo", average="macro")
    roc_auc["weighted"] = roc_auc_score(y_test, predict, multi_class="ovo", average="weighted")
    # ===================================================

    y_pre = np.argmax(predict, axis=1)
    y_test = np.argmax(y_test, axis=1)

    report = classification_report(y_test, y_pre, output_dict=True)
    df1 = pd.DataFrame(report).transpose()
    
    df1['auc'] = [roc_auc[0], roc_auc[1], roc_auc[2], " ", roc_auc["macro"], roc_auc["weighted"]]
    
    # Write it into csv format
    #df1.to_csv(path+'report.csv', index=True, header=True)
    
    cvpre.append(df1.loc['macro avg','precision'] * 100)
    cvrecall.append(df1.loc['macro avg','recall'] * 100)
    cvf1.append(df1.loc['macro avg','f1-score'] * 100)
    cvauc.append(df1.loc['macro avg','auc'] * 100)
    
#     data = confusion_matrix(y_test, y_pre)
#     names = ['normal', 'pneumonia', 'COVID-19']
#     df_cm = pd.DataFrame(data, columns=names, index=names)
#     df_cm.to_csv(path+'cm.csv', index=True, header=True)
    print("========================= 第"+ str(k+1) +"折结束 ============================")
    
print("accuracy：%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("precision：%.2f%% (+/- %.2f%%)" % (np.mean(cvpre), np.std(cvpre)))
print("recall：%.2f%% (+/- %.2f%%)" % (np.mean(cvrecall), np.std(cvrecall)))
print("f1-score：%.2f%% (+/- %.2f%%)" % (np.mean(cvf1), np.std(cvf1)))
print("auc：%.2f%% (+/- %.2f%%)" % (np.mean(cvauc), np.std(cvauc)))

In [ ]:
X = np.load("newData/image.npy")
Y = np.load("newData/label.npy")

In [ ]:
predict = model.predict([X])

y = label_binarize(Y, classes=[0, 1, 2])
n_classes = y.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y[:, i], predict[:, i], )
    roc_auc[i] = auc(fpr[i], tpr[i])
roc_auc["macro"] = roc_auc_score(y, predict, multi_class="ovo", average="macro")
roc_auc["weighted"] = roc_auc_score(y, predict, multi_class="ovo", average="weighted")


y_pre = np.argmax(predict, axis=1)
report = classification_report(Y, y_pre, output_dict=True)
df1 = pd.DataFrame(report).transpose()
df1['auc'] = [roc_auc[0], roc_auc[1], roc_auc[2], '', roc_auc["macro"], roc_auc["weighted"]]
df1